In [130]:
import pandas as pd
import numpy as np
from utils import spliter
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [131]:
data = pd.read_csv('utils/filtered_CNN_features_dataset.csv')
data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048
0,0,0.266585,1.337187,1.352905,0.100848,1.880685,0.059888,2.773605,0.115643,0.138749,...,0.039434,0.071662,0.733759,1.643910,0.041306,1.488236,0.048181,0.247783,0.300232,Paul_Bremer
1,1,0.355948,0.750634,0.909794,0.188214,0.680066,0.221442,3.089571,0.342022,0.087186,...,0.093867,0.089694,0.859610,1.764073,0.250430,1.133540,0.004091,0.153542,0.236034,Paul_Bremer
2,2,0.267111,1.015316,1.154480,0.131544,1.066389,0.024949,3.701925,0.145037,0.082419,...,0.080086,0.089589,0.663708,1.854499,0.078751,1.240009,0.065222,0.105058,0.227633,Paul_Bremer
3,3,0.260295,0.882900,0.885955,0.106698,2.663052,0.057836,1.854394,0.209269,0.103750,...,0.181255,0.182038,0.397535,1.222931,0.007285,0.878194,0.016247,0.110492,0.129523,Paul_Bremer
4,4,0.347203,0.623839,0.817085,0.212599,1.026321,0.176634,2.486715,0.426571,0.345026,...,0.267895,0.136346,0.713298,0.978184,0.059828,1.008205,0.042905,0.028597,0.199120,Paul_Bremer


In [132]:
data = data.drop(columns=['Unnamed: 0'])
data.head()

,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048
0,0.266585,1.337187,1.352905,0.100848,1.880685,0.059888,2.773605,0.115643,0.138749,0.056121,...,0.039434,0.071662,0.733759,1.643910,0.041306,1.488236,0.048181,0.247783,0.300232,Paul_Bremer
1,0.355948,0.750634,0.909794,0.188214,0.680066,0.221442,3.089571,0.342022,0.087186,0.234741,...,0.093867,0.089694,0.859610,1.764073,0.250430,1.133540,0.004091,0.153542,0.236034,Paul_Bremer
2,0.267111,1.015316,1.154480,0.131544,1.066389,0.024949,3.701925,0.145037,0.082419,0.253356,...,0.080086,0.089589,0.663708,1.854499,0.078751,1.240009,0.065222,0.105058,0.227633,Paul_Bremer
3,0.260295,0.882900,0.885955,0.106698,2.663052,0.057836,1.854394,0.209269,0.103750,0.149302,...,0.181255,0.182038,0.397535,1.222931,0.007285,0.878194,0.016247,0.110492,0.129523,Paul_Bremer
4,0.347203,0.623839,0.817085,0.212599,1.026321,0.176634,2.486715,0.426571,0.345026,0.375072,...,0.267895,0.136346,0.713298,0.978184,0.059828,1.008205,0.042905,0.028597,0.199120,Paul_Bremer


In [133]:
data.shape

(4324, 2049)

In [134]:
label_encoder = LabelEncoder()
data['2048'] = label_encoder.fit_transform(data['2048'])

In [135]:
x_train, x_test, y_train, y_test = spliter.train_test_spliter(data, target_col='2048', train_size = 0.9)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

/home/mayank/all_work/ML-DL/Face_Identification/Face_Detection/applying_ML_algorithms/utils/spliter.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  train = pd.concat([train, train_subset])
/home/mayank/all_work/ML-DL/Face_Identification/Face_Detection/applying_ML_algorithms/utils/spliter.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  test = pd.concat([test, test_subset])


((3813, 2048), (511, 2048), (3813,), (511,))

In [136]:
y_train = y_train.astype(np.float32).reshape(-1, 1)
y_test = y_test.astype(np.float32).reshape(-1, 1)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

onehothotencoder = OneHotEncoder()
y_train = onehothotencoder.fit_transform(y_train).toarray()
y_test = onehothotencoder.transform(y_test).toarray()

y_train.shape

(3813, 158)

In [137]:
import numpy as np

# Part 1: Function Implementations

def sigmoid(z):
    return 1/(1+np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z)*(1-sigmoid(z))

def relu(z):
    return np.maximum(0, z)

def softmax(z):
    z = z - np.max(z, axis=1, keepdims=True)  # numerical stability
    z = np.clip(z, -50, 50)  # limit extreme values
    exp_z = np.exp(z)
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def relu_derivative(z):
    return (z>0)*1

def linear(z):
    return z

def linear_derivative(z):
    return 1

def mean_squared_error(y_true, y_pred):
    return np.mean(np.square(y_true-y_pred))

def cross_entropy_loss(y_true, y_pred):
    eps = 1e-15  # avoid log(0)
    y_pred = np.clip(y_pred, eps, 1 - eps)
    return -np.mean(np.sum(y_true * np.log(y_pred), axis=1))

def activation_decider(activation, z):
  if activation == 'sigmoid':
    return sigmoid(z)
  elif activation == 'relu':
    return relu(z)
  elif activation == 'linear':
    return linear(z)
  elif activation == 'softmax':
    return softmax(z)

def activation_derivative_decider(activation, z):
  if activation == 'sigmoid':
    return sigmoid_derivative(z)
  elif activation == 'relu':
    return relu_derivative(z)
  elif activation == 'linear':
    return linear_derivative(z)

In [138]:
class NeuralNetwork:

    def __init__(self, layers, activations):
        self.layers = layers
        self.activations = activations
        self.weights = []
        self.biases = []

        for i in range(len(layers) - 1):
            # He initialization for ReLU, Glorot otherwise
            if activations[i] == 'relu':
                stddev = np.sqrt(2 / layers[i])  # He initialization
            else:
                stddev = np.sqrt(1 / (layers[i] + layers[i + 1]))  # Glorot/Xavier
            self.weights.append(np.random.randn(layers[i], layers[i + 1]) * stddev)
            self.biases.append(np.zeros((1, layers[i + 1])))

    def forward(self, x):
        z_values = []
        activation_values = [x]

        for i in range(len(self.weights)):
            z = np.dot(x, self.weights[i]) + self.biases[i]
            z_values.append(z)
            x = activation_decider(self.activations[i], z)
            activation_values.append(x)

        return z_values, activation_values

    def backward(self, x, y, activations, z_values, learning_rate):
        m = x.shape[0]
        deltas = []

        # Output layer delta
        if self.activations[-1] == 'softmax':
            delta = activations[-1] - y  # softmax + cross-entropy
        else:
            delta = (activations[-1] - y) * activation_derivative_decider(self.activations[-1], z_values[-1])

        deltas.append(delta)

        # Backpropagate through hidden layers
        for i in reversed(range(len(self.weights) - 1)):
            delta = deltas[-1].dot(self.weights[i + 1].T) * activation_derivative_decider(self.activations[i], z_values[i])
            deltas.append(delta)

        deltas.reverse()

        # Update weights and biases
        for i in range(len(self.weights)):
            a_prev = x if i == 0 else activations[i]
            grad_w = a_prev.T.dot(deltas[i]) / m
            grad_b = np.mean(deltas[i], axis=0, keepdims=True)
            grad_w = np.clip(grad_w, -1.0, 1.0)
            grad_b = np.clip(grad_b, -1.0, 1.0)
            self.weights[i] -= learning_rate * grad_w
            self.biases[i] -= learning_rate * grad_b

    def train(self, x_train, y_train, epochs, learning_rate):
        for epoch in range(epochs):
            z_values, activations = self.forward(x_train)
            self.backward(x_train, y_train, activations, z_values, learning_rate)
            if (epoch + 1)%100 == 0:
                loss = cross_entropy_loss(y_train, activations[-1])
                print(f"Epoch {epoch+1}: Loss = {loss:.6f}")

    def predict(self, x):
        _, activations = self.forward(x)
        return np.argmax(activations[-1], axis=1)  # class prediction


In [139]:
input_size = x_train.shape[1]
num_classes = len(data['2048'].unique())

nn = NeuralNetwork([input_size, 512, 256, 128, num_classes], activations = ['relu', 'relu', 'relu', 'softmax'])
nn.train(x_train, y_train, epochs=1000, learning_rate=0.1)

predictions = nn.predict(x_train)
acc = np.mean(predictions == y_train.argmax(axis=1))
print(f"Train Accuracy : {acc}")

predictions = nn.predict(x_test)
acc = np.mean(predictions == y_test.argmax(axis=1))
print(f"Test Accuracy : {acc}")

Epoch 100: Loss = 1.613926
Epoch 200: Loss = 0.818150
Epoch 300: Loss = 0.104134
Epoch 400: Loss = 0.046099
Epoch 500: Loss = 0.027508
Epoch 600: Loss = 0.019040
Epoch 700: Loss = 0.014332
Epoch 800: Loss = 0.011377
Epoch 900: Loss = 0.009366
Epoch 1000: Loss = 0.007919
Train Accuracy : 1.0
Test Accuracy : 0.44227005870841485


In [141]:
counts = data['2048'].value_counts()
filter_labels = counts[counts > 80].index
data = data[data['2048'].isin(filter_labels)]
data['2048'].value_counts()

2048
35     530
23     236
149    144
29     121
36     109
Name: count, dtype: int64

In [142]:
x_train, x_test, y_train, y_test = spliter.train_test_spliter(data, target_col='2048', train_size = 0.9)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

/home/mayank/all_work/ML-DL/Face_Identification/Face_Detection/applying_ML_algorithms/utils/spliter.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  train = pd.concat([train, train_subset])
/home/mayank/all_work/ML-DL/Face_Identification/Face_Detection/applying_ML_algorithms/utils/spliter.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  test = pd.concat([test, test_subset])


((1024, 2048), (116, 2048), (1024,), (116,))

In [144]:
lda = LDA(n_components=4)
lda.fit(x_train, y_train.astype(np.float32))
x_train_lda = lda.transform(x_train)
x_test_lda = lda.transform(x_test)

In [145]:
y_train = y_train.astype(np.float32).reshape(-1, 1)
y_test = y_test.astype(np.float32).reshape(-1, 1)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

onehothotencoder = OneHotEncoder()
y_train = onehothotencoder.fit_transform(y_train).toarray()
y_test = onehothotencoder.transform(y_test).toarray()

y_train.shape

(1024, 5)

In [ ]:
input_size = x_train.shape[1]
num_classes = len(data['2048'].unique())

nn = NeuralNetwork([input_size, 512, 256, 128, num_classes], activations = ['relu', 'relu', 'relu', 'softmax'])
nn.train(x_train, y_train, epochs=1000, learning_rate=0.1)

predictions = nn.predict(x_train)
acc = np.mean(predictions == y_train.argmax(axis=1))
print(f"Train Accuracy : {acc}")

predictions = nn.predict(x_test)
acc = np.mean(predictions == y_test.argmax(axis=1))
print(f"Test Accuracy : {acc}")

In [187]:
input_size = x_train_lda.shape[1]
num_classes = len(data['2048'].unique())

nn = NeuralNetwork([input_size, 32, 16, num_classes], activations = ['relu', 'relu', 'softmax'])
nn.train(x_train_lda, y_train, epochs=2000, learning_rate=0.01)

predictions = nn.predict(x_train_lda)
acc = np.mean(predictions == y_train.argmax(axis=1))
print(f"Train Accuracy : {acc}")

predictions = nn.predict(x_test_lda)
acc = np.mean(predictions == y_test.argmax(axis=1))
print(f"Test Accuracy : {acc}")

Epoch 100: Loss = 0.073208
Epoch 200: Loss = 0.038483
Epoch 300: Loss = 0.027264
Epoch 400: Loss = 0.021565
Epoch 500: Loss = 0.018062
Epoch 600: Loss = 0.015673
Epoch 700: Loss = 0.013933
Epoch 800: Loss = 0.012607
Epoch 900: Loss = 0.011532
Epoch 1000: Loss = 0.010638
Epoch 1100: Loss = 0.009913
Epoch 1200: Loss = 0.009323
Epoch 1300: Loss = 0.008818
Epoch 1400: Loss = 0.008380
Epoch 1500: Loss = 0.007996
Epoch 1600: Loss = 0.007656
Epoch 1700: Loss = 0.007351
Epoch 1800: Loss = 0.007076
Epoch 1900: Loss = 0.006826
Epoch 2000: Loss = 0.006598
Train Accuracy : 0.9990234375
Test Accuracy : 0.7672413793103449
